# Package import

In [2]:
from collections import OrderedDict
from time import time
import copy as cp
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm
import seaborn
import theano as thno
import theano.tensor as T
import itertools as itr
import pickle as pkl
from tqdm import tqdm

from scipy import integrate
from scipy.optimize import fmin_powell
from scipy.stats import binom

# Utilities

In [4]:
### log transformation of variables in the hear disease data
def log_transform(data, center = False, scale = False):
    data.loc[:, data.columns == 'age'] = np.log(data.loc[:, data.columns == 'age'])
    data.loc[:, data.columns == 'trestbps'] = np.log(data.loc[:, data.columns == 'trestbps'])
    data.loc[:, data.columns == 'chol'] = np.log(data.loc[:, data.columns == 'chol'])
    data.loc[:, data.columns == 'thalach'] = np.log(data.loc[:, data.columns == 'thalach'])
    data.loc[:, data.columns == 'oldpeak'] = np.log(data.loc[:, data.columns == 'oldpeak'])
    if center:
        data.loc[:, data.columns == 'age'] = data.loc[:, data.columns == 'age']-  data.loc[:, data.columns == 'age'].mean()
        data.loc[:, data.columns == 'trestbps'] = data.loc[:, data.columns == 'trestbps'] - data.loc[:, data.columns == 'trestbps'].mean()
        data.loc[:, data.columns == 'chol'] = data.loc[:, data.columns == 'chol'] - data.loc[:, data.columns == 'chol'].mean()
        data.loc[:, data.columns == 'thalach'] = data.loc[:, data.columns == 'thalach'] - data.loc[:, data.columns == 'thalach'].mean()
        data.loc[:, data.columns == 'oldpeak'] = data.loc[:, data.columns == 'oldpeak'] - data.loc[:, data.columns == 'oldpeak'].mean()
    if scale:
        data.loc[:, data.columns != 'target'] = data.loc[:, data.columns != 'target'] / data.loc[:, data.columns != 'target'].std() 
    return data

def power_set_index(n_predictors):
    """Generates a matrix of variable indicators defining the space of 
    all the models
    
    Args:
        n_predictors: Number of predictors to be considered
    Returns:
        A matrix with variable indicators"""
    power_set  = itr.product([0,1], repeat = n_predictors)
    array_index = []
    for i in list(power_set):
        array_index = array_index + [np.array(i)]
    array_index = np.array(array_index)
    ids = np.array([i for i in range(len(array_index))])
    return np.append( np.append(ids[:,None], np.ones(len(array_index))[:, None], axis = 1), array_index, axis = 1)

def evidence_integral(n_mc, n_mc_evidence, data_evidence, sigma):
    """MC approximation of the evidence integral"""
    log_evidence = np.zeros(n_mc)
    # likelihood eval
    for i in tqdm(range(n_mc), desc = "log likelihood eval"):
        np.random.normal(0, sigma, len(variables))
        z = np.sum(data_evidence * np.random.normal(0, sigma, len(variables)), axis = 1)
        p = 1 / (1 + np.exp(- z))
        log_evidence[i] = np.sum(binom.logpmf(k=data["target"].values, n=1, p = p))

    mc_integral = np.empty(n_mc_evidence)
    for i in tqdm(range(n_mc_evidence), desc = "Integral calc"):
        j = (i + n_mc - n_mc_evidence)
        m = max(log_evidence[:(j + 1)])
        mc_integral[i] = (np.exp(m) * np.sum(np.exp(log_evidence[:(j + 1)] - m))) / (j + 1)
    return log_evidence, mc_integral

# MCMC psoterior approximation and MC evidence computaiton

In [7]:
### Loading and pre-processing data
np.random.seed(0)
heart = pd.read_csv("heart.csv", index_col= None)

data_raw = cp.deepcopy(heart[~pd.isnull(heart["target"])])
variables_set = ["target", "chol", "trestbps", "sex"]
K = len(variables_set) - 1 # Number of predictors
model_space = power_set_index(K).astype(int)

### MCMC approximation and MC integration for a single model
model_index = 0
for model in [model_index]:
    variables = np.array(variables_set)[model_space[model,1:].astype(bool)]
    data = cp.deepcopy(log_transform(data_raw[variables], center = True))

    # MCMC
    sigma = float(3.0) # Standard deviation for the prior distribution of regression params
    n_sample = 20000
    n_tune = 10000
    sampler = "NUTS"
    model_string = "target ~ 1"
    for i in range(len(variables) - 1):
        model_string = model_string + '+' + variables[i+1]
        
    with pm.Model() as logistic_model:
        priors = {
            "Intercept": pm.Normal.dist(mu = 0, sigma = sigma)
        }
        for var in variables[1:]:
            priors[var] = pm.Normal.dist(mu = 0, sigma = sigma)

        pm.glm.GLM.from_formula(
            model_string, data, family=pm.glm.families.Binomial(), priors=priors
        )
        if sampler == "NUTS":
            trace = pm.sample(n_sample, tune=n_tune, init="adapt_diag", chains = 1)
        elif sampler == "MH":
            step = pm.Metropolis()
            trace = pm.sample(50000, tune=5000, chains = 1, step = step)
        pm.backends.ndarray.save_trace(trace, "trace_logistic_" + str(n_tune) + "_" + str(n_sample) + "_sigma_" + str(sigma) + "_model_" + str(model) + "_sampler_" + sampler, overwrite=True)

    # MC    
    n_mc = 100000 # Number of MC samples used for evidence computation
    n_mc_evidence = 10000 # Number of samples stored
    data_evidence = cp.deepcopy(data.values)
    data_evidence[:,0] = np.ones(len(data))
    log_evidence, mc_integral = evidence_integral(n_mc, n_mc_evidence, data_evidence, sigma)
    evidence_results = {"log_evidence": log_evidence, "mc_integral": mc_integral}
    with open("evidence_sigma_" + str(sigma) + "_model_" + str(model) + "_nmc_" + str(n_mc) + '.pickle', 'wb') as handle:
        pkl.dump(evidence_results , handle, protocol=pkl.HIGHEST_PROTOCOL)

Auto-assigning NUTS sampler...
Initializing NUTS using adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [Intercept]
Sampling chain 0, 0 divergences: 100%|█████████████████████████████████████████| 30000/30000 [00:19<00:00, 1572.10it/s]
Only one chain was sampled, this makes it impossible to run some convergence checks
Integral calc: 100%|█████████████████████████████████████████████████████████████| 10000/10000 [01:53<00:00, 88.32it/s]
